In [1]:
import numpy as np
import netCDF4 as nc
import glob
import matplotlib.pyplot as plt
import re
import pandas as pd
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
file_path = '/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/radar/'

file_list = sorted(glob.glob('/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/radar/2023*.nc'))  # Adjust path based on month


In [ ]:
mmdd_list = []
radarAccum_list = []
radarAccumUpp_list = []
radarAccumLow_list = []

i = 0
for file in file_list[:]:
    dayFile = nc.Dataset(file)

    dbz = np.array(dayFile.variables['Zh'][:, 0])
    dbz[dbz > 1000] = np.nan
    z = 10 ** (dbz / 10)

    a = 18
    b = 1.1
    aLow = 11
    aHigh = 43

    precip = (z / a) ** (1 / b)
    precipLow = (z / aHigh) ** (1 / b)
    precipHigh = (z / aLow) ** (1 / b)

    radarAccum = np.trapz(precip[~np.isnan(precip)], dx=24/len(precip))
    radarAccumHigh = np.trapz(precipHigh[~np.isnan(precipHigh)], dx=24/len(precipHigh))
    radarAccumLow = np.trapz(precipLow[~np.isnan(precipLow)], dx=24/len(precipLow))
    
    # Regular expression to find MMDD after '2023'
    match = re.search(r'/2023(\d{4})', file)
    mmdd = match.group(1)

    mmdd_list.append(mmdd)
    radarAccum_list.append(radarAccum)
    radarAccumUpp_list.append(radarAccumHigh)
    radarAccumLow_list.append(radarAccumLow)
    if i % 10 == 0:
        print(i)
    i = i + 1




/tmp/ipykernel_4186618/60221649.py:23: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  radarAccum = np.trapz(precip[~np.isnan(precip)], dx=24/len(precip))
/tmp/ipykernel_4186618/60221649.py:24: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  radarAccumHigh = np.trapz(precipHigh[~np.isnan(precipHigh)], dx=24/len(precipHigh))
/tmp/ipykernel_4186618/60221649.py:25: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  radarAccumLow = np.trapz(precipLow[~np.isnan(precipLow)], dx=24/len(precipLow))


0


In [ ]:
df = pd.DataFrame({'mmdd': mmdd_list, 'radarAccum': radarAccum_list})
df.to_csv('accumulation_by_day.csv', index=False)

In [ ]:
# [:, 0] is radar
# 1 is disdrometer
# 2, 3 are era5 lower and upper bounds
allAccum = np.empty([365, 4])
allAccum[:, :] = np.nan

In [ ]:
radarAccumArr = np.array(radarAccum_list)
mmddArr = np.array(mmdd_list)

for i in range(len(mmddArr)):
    date_obj = datetime.strptime('2023' + mmddArr[i], '%Y%m%d')
    day_of_year_idx = date_obj.timetuple().tm_yday - 1  # 0-based indexing

    allAccum[day_of_year_idx, 0] = radarAccumArr[i]

In [ ]:
print(np.sum(radarAccumArr))

In [ ]:
dis_file_path = '/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/disdrometer/'

dis_file_list = sorted(glob.glob('/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/disdrometer/2023*.nc'))  # Adjust path based on month

dis_mmdd_list = []
disAccum_list = []

for dis_file in dis_file_list:
    dayFile = nc.Dataset(dis_file)

    rainRate = np.array(dayFile.variables['rainfall_rate']) * 1000 * 60 * 60 # time units
    snowRate = (rainRate ** 1.9) * 0.03344 # Matrosov et al (2022) mass size relation

    disdAccum = np.trapz(snowRate, dx=24/len(snowRate))
    
    # Regular expression to find MMDD after '2023'
    match = re.search(r'/2023(\d{4})', dis_file)
    mmdd = match.group(1)

    dis_mmdd_list.append(mmdd)
    disAccum_list.append(disdAccum)




In [ ]:
disdf = pd.DataFrame({'mmdd': dis_mmdd_list, 'disAccum': disAccum_list})
disdf.to_csv('disdrometer_accumulation_by_day.csv', index=False)

In [ ]:
disAccumArr = np.array(disAccum_list)
dismmddArr = np.array(dis_mmdd_list)

for i in range(len(dismmddArr)):
    date_obj = datetime.strptime('2023' + dismmddArr[i], '%Y%m%d')
    day_of_year_idx = date_obj.timetuple().tm_yday - 1  # 0-based indexing

    allAccum[day_of_year_idx, 1] = disAccumArr[i]

In [ ]:
print(np.sum(disAccumArr))

In [ ]:
plt.scatter(allAccum[:, 0], allAccum[:, 1])
plt.ylim(0, 30)
plt.title("Radar vs Disdrometer, Disdrometer y limits")

In [ ]:
plt.scatter(allAccum[:, 0], allAccum[:, 1])
plt.title("Radar vs Disdrometer, no limits")

In [ ]:
#file_path = '/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/radar/20230426_neumayer_mira.nc'
modfile_path = '/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/era_precip/a5fe22a8557d37e66a32d5412c5f2411.nc'
model = nc.Dataset(modfile_path)

# Coords of Neumayer III
minMod = np.array(model.variables['mntpr'][:, 23, 87]) * 60 * 60
maxMod = np.array(model.variables['mxtpr'][:, 23, 87]) * 60 * 60

print(model)

In [ ]:
eraUpperAccum = np.trapz(maxMod)
eraLowerAccum = np.trapz(minMod)

print(eraUpperAccum)
print(eraLowerAccum)

In [ ]:
accum_daily_low = np.sum(minMod.reshape(365, 24), axis=1)
accum_daily_high = np.sum(maxMod.reshape(365, 24), axis=1)


In [ ]:
dates = np.arange('2023-01-01', '2024-01-01', dtype='datetime64[D]')
plt.plot(dates, allAccum[:, 0], label='radar')
#plt.plot(dates, allAccum[:, 1], label='disdrometer')
plt.plot(dates, accum_daily_low, label='era5 lower')
plt.plot(dates, accum_daily_high, label='era5 upper')


plt.legend()


In [ ]:
dates = np.arange('2023-01-01', '2024-01-01', dtype='datetime64[D]')
plt.plot(dates, allAccum[:, 0], label='radar')
#plt.plot(dates, allAccum[:, 1], label='disdrometer')
plt.plot(dates, accum_daily_low, label='era5 lower')
plt.plot(dates, accum_daily_high, label='era5 upper')

plt.legend()


In [ ]:
plt.scatter(allAccum[:, 0], accum_daily_high)
plt.ylabel('era5')
plt.xlabel('radar')
plt.plot(np.arange(0, 30), np.arange(0, 30), c='green')
#plt.gca().set_xscale("log")
#plt.gca().set_yscale("log")

plt.title("Radar (A=18) vs ERA5")

In [ ]:
plt.scatter(allAccum[:, 0], accum_daily_high)
plt.ylabel('era5')
plt.xlabel('radar')
plt.xlim(0, 5)
plt.ylim(0, 5)
plt.plot(np.arange(0, 30), np.arange(0, 30), c='green')
plt.title("Radar vs ERA5")

In [ ]:
plt.scatter(allAccum[:, 1], accum_daily_high)
plt.ylabel('era5')
plt.xlabel('disdrometer')
plt.xlim(0, 5)
plt.ylim(0, 5)
plt.plot(np.arange(0, 30), np.arange(0, 30), c='green')
plt.title("Disdrometer vs ERA5")

In [ ]:
plt.scatter(allAccum[:, 0], allAccum[:, 1])
plt.ylabel('disdrometer')
plt.xlabel('radar')
plt.xlim(0, 5)
plt.ylim(0, 5)
plt.plot(np.arange(0, 30), np.arange(0, 30), c='green')
plt.title("Disdrometer vs Radar")

In [ ]:
datesPD = pd.date_range('2023-01-01', periods=365, freq='D')

mmdd_new = datesPD.strftime('%m%d')

# Build the DataFrame
df = pd.DataFrame({
    'mmdd': mmdd_new,
    'accum': accum_daily_low
})


# Save to CSV
df.to_csv('era5_accumulation_low.csv', index=False)

In [ ]:
print("Radar Accumulation: " + str(np.sum(radarAccumArr)) + " mm")
print("Disdrometer Accumulation: " + str(np.sum(disAccumArr)) + " mm")
print("ERA5 Accumulation Lower: " + str(eraLowerAccum) + " mm")
print("ERA5 Accumulation Upper: " + str(eraUpperAccum) + " mm")

In [ ]:
# An apples-to-apples: only days when there's radar data

eraUpperApp = np.trapz(accum_daily_high[~np.isnan(allAccum[:, 0])])
eraLowerApp = np.trapz(accum_daily_low[~np.isnan(allAccum[:, 0])])

print("ERA5 Accumulation Lower: " + str(eraLowerApp) + " mm")
print("ERA5 Accumulation Upper: " + str(eraUpperApp) + " mm")

In [ ]:
# cross plots with hex bin - adds an intensity color for point density

plt.hexbin(allAccum[:, 0], accum_daily_high, gridsize=15, bins='log')
plt.xlabel('Radar')
plt.ylabel('ERA5')

In [ ]:
dates = np.arange('2023-01-01', '2024-01-01', dtype='datetime64[D]')
plt.figure(figsize=(20, 10))
plt.plot(dates, allAccum[:, 0], label='radar (A=18)')
#plt.plot(dates, allAccum[:, 1], label='disdrometer')
plt.plot(dates, (accum_daily_low+accum_daily_high)/2, label='ERA5')
plt.ylabel("Daily snow accumulation (mm)")
#plt.plot(dates, accum_daily_high, label='era5 upper')


plt.legend()


In [ ]:
import seaborn as sns

data = [allAccum[:, 0], (accum_daily_low+accum_daily_high)/2]

sns.violinplot(data=data)
plt.xticks([0, 1], ['Radar', 'ERA5'])
plt.title("Violin Plot")
plt.ylabel("Daily accumulation")
plt.show()

In [ ]:
a = allAccum[:, 0]
b = (accum_daily_low+accum_daily_high)/2
data = [a[np.where(a < 1)], b[np.where(b < 1)]]

sns.violinplot(data=data)
plt.xticks([0, 1], ['Radar', 'ERA5'])
plt.title("Violin Plot, days < 1 mm")
plt.ylabel("Daily accumulation")
plt.show()

In [ ]:
a = allAccum[:, 0]
b = (accum_daily_low+accum_daily_high)/2
data = [a[np.where(a > 1)], b[np.where(b > 1)]]

sns.violinplot(data=data)
plt.xticks([0, 1], ['Radar', 'ERA5'])
plt.title("Violin Plot, days > 1 mm")
plt.ylabel("Daily accumulation")
plt.show()

In [ ]:
newMask = ~np.isnan(a) & ~np.isnan(b)

# Step 2: Apply mask to get valid pairs
a_valid = a[newMask]
b_valid = b[newMask]

# Step 3: Compute metrics
mae = mean_absolute_error(a_valid, b_valid)
mse = mean_squared_error(a_valid, b_valid)
rmse = np.sqrt(mse)
bias = np.mean(a_valid - b_valid)
corr = np.corrcoef(a_valid, b_valid)[0, 1]
r2 = r2_score(a_valid, b_valid)

# Step 4: Print results
print("Comparison Metrics:")
print(f"  MAE : {mae:.3f}")
print(f"  RMSE: {rmse:.3f}")
print(f"  Bias: {bias:.3f}")
print(f"  Corr: {corr:.3f}")
print(f"  R²  : {r2:.3f}")

In [3]:
data = np.genfromtxt('accumulation_by_day.csv', delimiter=',', skip_header=1)

mmdd = data[:, 0].astype(int)
radarAccum = data[:, 1]

In [31]:
df1 = pd.read_csv('accumulation_by_day.csv')
df2 = pd.read_csv('accumulation_sued_cleaned_interpolated.csv')

df2['date'] = pd.to_datetime(df2['date'])

df2_2023 = df2[df2['date'].dt.year == 2023]

dates1 = df1['mmdd'].values  # NumPy array of datetime64[ns]
radarAccum = df1['radarAccum'].values

dates2 = df2_2023['date'].values
cumulative_accumStake = (df2_2023['cumulative_accumulation'].values - 3176.7) * 10

In [32]:
df1_cp = pd.read_csv('accumulation_by_day.csv')
df1_cp['date'] = pd.to_datetime('2023' + df1_cp['mmdd'].astype(str).str.zfill(4), format='%Y%m%d')
df1_cp = df1_cp.set_index('date')

# Group by month and sum
radarAccum_cumulative = df1_cp['radarAccum'].resample('M').sum()

/tmp/ipykernel_4187467/2013174266.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  radarAccum_cumulative = df1_cp['radarAccum'].resample('M').sum()


In [33]:
cumulative_radarAccum = radarAccum_cumulative.cumsum()

In [46]:
finalRadarAccum_mid = np.array(cumulative_radarAccum-cumulative_radarAccum[0])
finalRadarAccum_high = np.array(cumulative_radarAccum-cumulative_radarAccum[0]) * 1.565
finalRadarAccum_low = np.array(cumulative_radarAccum-cumulative_radarAccum[0]) * 0.453

/tmp/ipykernel_4187467/3031568167.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  finalRadarAccum_mid = np.array(cumulative_radarAccum-cumulative_radarAccum[0])
/tmp/ipykernel_4187467/3031568167.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  finalRadarAccum_high = np.array(cumulative_radarAccum-cumulative_radarAccum[0]) * 1.565
/tmp/ipykernel_4187467/3031568167.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  finalRadarA

In [48]:
finalStake = np.array(cumulative_accumStake-cumulative_accumStake[0])

In [50]:
df3 = pd.read_csv('era5_accumulation_high.csv')
df3['date'] = pd.to_datetime('2023' + df3['mmdd'].astype(str).str.zfill(4), format='%Y%m%d')
df3 = df3.set_index('date')

df4 = pd.read_csv('era5_accumulation_low.csv')
df4['date'] = pd.to_datetime('2023' + df4['mmdd'].astype(str).str.zfill(4), format='%Y%m%d')
df4 = df4.set_index('date')

# Group by month and sum
era5High_cumulative = df3['accum'].resample('M').sum()
era5Low_cumulative = df4['accum'].resample('M').sum()

cumulative_eraHigh = era5High_cumulative.cumsum()
cumulative_eraLow = era5Low_cumulative.cumsum()

finalEraHigh = np.array(cumulative_eraHigh-cumulative_eraHigh[0])
finalEraLow = np.array(cumulative_eraLow-cumulative_eraLow[0])

/tmp/ipykernel_4187467/2213756620.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  era5High_cumulative = df3['accum'].resample('M').sum()
/tmp/ipykernel_4187467/2213756620.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  era5Low_cumulative = df4['accum'].resample('M').sum()
/tmp/ipykernel_4187467/2213756620.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  finalEraHigh = np.array(cumulative_eraHigh-cumulative_eraHigh[0])
/tmp/ipykernel_4187467/2213756620.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc

In [52]:
print(finalEraLow)

[  0.           3.84135249  53.08885491 176.4691818  212.76483508
 312.88247049 364.4748681  398.98867507 453.51219088 608.17565935
 654.93106859 677.09770196]
